In [1]:
import Pkg;
Pkg.activate(@__DIR__);
Pkg.instantiate();

  Activating environment at `~/Git/solvers/tinympc/tinympc-dev-julia/quat_mpc_state/Project.toml`


In [2]:
using LinearAlgebra
using BlockDiagonals
using ForwardDiff
using Plots
using Random;
using Printf

using RobotZoo: Quadrotor
using RobotDynamics
using MeshCat
using ColorTypes
using TrajOptPlots
using GeometryBasics: Point, Cylinder
using CoordinateTransformations
using Rotations

include("../quaternion-stuff.jl")

E (generic function with 1 method)

In [3]:
#Quadrotor parameters
m = 0.035
ℓ = 0.046
J = [16.6e-6 0.83e-6 0.72e-6
    0.83e-6 16.6e-6 1.8e-6
    0.72e-6 1.8e-6 29.3e-6];
g = 9.81
# thrustToTorque = 0.005964552
thrustToTorque = 0.0008
ℓ = 0.046 / sqrt(2)
scale = 65535
kt = 2.245365e-6 * scale # u is PWM in range [0...1]
km = kt * thrustToTorque #4.4733e-8

freq1 = 50
h = 1 / freq1 #50 Hz

nx = 12     # number of states (linearized)
nu = 4     # number of controls

4

In [4]:
function quad_dynamics(x, u)
  r = x[1:3]
  q = x[4:7] / norm(x[4:7]) #normalize q just to be careful
  v = x[8:10]
  ω = x[11:13]
  Q = qtoQ(q)

  ṙ = v
  q̇ = 0.5 * L(q) * H * ω

  v̇ = [0; 0; -g] + (1 / m) * Q * [zeros(2, 4); kt * ones(1, 4)] * u

  ω̇ = J \ (-hat(ω) * J * ω + [-ℓ*kt -ℓ*kt ℓ*kt ℓ*kt; -ℓ*kt ℓ*kt ℓ*kt -ℓ*kt; -km km -km km] * u)

  return [ṙ; q̇; v̇; ω̇]
end

function quad_dynamics_rk4(x, u, h)
  #RK4 integration with zero-order hold on u
  f1 = quad_dynamics(x, u)
  f2 = quad_dynamics(x + 0.5 * h * f1, u)
  f3 = quad_dynamics(x + 0.5 * h * f2, u)
  f4 = quad_dynamics(x + h * f3, u)
  xn = x + (h / 6.0) * (f1 + 2 * f2 + 2 * f3 + f4)
  xn[4:7] .= xn[4:7] / norm(xn[4:7]) #re-normalize quaternion
  return xn
end

quad_dynamics_rk4 (generic function with 1 method)

In [5]:
# Goal state
@show uhover = (m * g / kt / 4) * ones(4)  # m = 30g and max thrust = 60g
rg = [0.0; 0; 0.0]
qg = [1.0; 0; 0; 0]
vg = zeros(3)
ωg = zeros(3)
xg = [rg; qg; vg; ωg];

uhover = (((m * g) / kt) / 4) * ones(4) = [0.5833333520642209, 0.5833333520642209, 0.5833333520642209, 0.5833333520642209]


In [6]:
# Linearize dynamics about hover
A = ForwardDiff.jacobian(x -> quad_dynamics_rk4(x, uhover, h), xg)
B = ForwardDiff.jacobian(u -> quad_dynamics_rk4(xg, u, h), uhover);
Ã = Array(E(qg)' * A * E(qg))
B̃ = Array(E(qg)' * B);

In [7]:
vis = Visualizer()
quad_model = Quadrotor()
TrajOptPlots.set_mesh!(vis, quad_model)

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8702
└ @ MeshCat /home/sam/.julia/packages/MeshCat/GlCMx/src/visualizer.jl:73


false

In [86]:
include("../LibTinyMPC.jl")

Tfinal = 5.0
NTOTAL = Int(Tfinal/h)+1

# Reference trajectory
X̃ref = [zeros(nx) for i = 1:NTOTAL]
scale = 5
# Xref = [[sin(2*t); 1.5*cos(t); 0.7*sin(3*t); 1; zeros(9)] for t in range(-pi/2*2, 3*pi/2*2, length=NTOTAL)] # mega pringle
# Xref = [[sin(scale*t)*cos(t); cos(scale*t)*cos(t); 0.75*sin(t); 1; zeros(9)] for t in range(0, 2*pi, length=NTOTAL)] # ball
# Xref = [[0.05; t; 0.5;   1; zeros(3);   zeros(6)] for t = range(0, 5, length = NTOTAL)]; # straight line
# Xref = [zeros(nx+1) for i = 1:NTOTAL] # origin

Xref = [[0.0; t; 0.5;   1; zeros(3);   zeros(6)] for t = range(0, 5, length = NTOTAL)]; # straight line with barrel roll
num_ref_points = length(Xref)
# start barrel roll at 2/5 and end at 3/5 of trajectory
start_barrel_roll_ratio = 2.0/5.0
end_barrel_roll_ratio = 3.0/5.0
num_point_in_barrel_roll = Int(floor(num_ref_points * (end_barrel_roll_ratio - start_barrel_roll_ratio)))
barrel_roll_angles = LinRange(0.0, 2*pi, num_point_in_barrel_roll) # create rough desired angles for barrel roll
quats = [[cos(α/2), 0, sin(α/2), 0] for α in barrel_roll_angles]

start_barrel_roll_index = Int(floor(num_ref_points*start_barrel_roll_ratio))
for i in 1:num_point_in_barrel_roll
    Xref[i+start_barrel_roll_index][4:7] = quats[i]
end

Uref = [zeros(nu) for i = 1:NTOTAL-1]

# Compute reference velocity from reference position
for i = 1:NTOTAL-1
    Xref[i][8:10] = (Xref[i+1][1:3] - Xref[i][1:3])/h
end

# Convert (13 state) Xref to reduced form (12 state) X̃ref
for k = 1:NTOTAL
    x = Xref[k]
    qx = x[4:7]
    ϕ = qtorp(L(qg)'*qx)
    X̃ref[k] = [x[1:3]; ϕ; x[8:10]; x[11:13]]
end

X̃ref = hcat(X̃ref...)
Uref = hcat(Uref...)

# Visualize reference
delete!(vis["xHistLine"])
delete!(vis["XrefLine"])
XrefLine = [Point(x_[1], x_[2], x_[3]) for x_ in Xref]
setobject!(vis["XrefLine"], Object(PointCloud(XrefLine), 
        LineBasicMaterial(color=Colors.RGBA(0.0, 0.45, 1.0)), "Line"))

max_dev_x = [0.001; 0.001; 0.001; 0.5; 0.5; 0.05; 0.5; 0.5; 0.5; 0.7; 0.7; 0.2] / 1
max_dev_u = [0.5; 0.5; 0.5; 0.5] / 10
Q = diagm(1 ./ (max_dev_x .^ 2))
R = diagm(1 ./ (max_dev_u .^ 2))

Nh = 100

solver = TinySolver(Ã, B̃, Q, R, Nh, X̃ref, Uref);

"ihlqr converged in 311 iterations"

In [ ]:
# Create more caches for different attitudes

xbar = RBState(quad_model, Xhist[:,1])
if quad_model.ned
    rq = position(xbar)
    vq = linear_velocity(xbar)
    rq = SA[rq[1],-rq[2],-rq[3]]
    vq = SA[vq[1],-vq[2],-vq[3]]
    xbar = RBState(rq, RotX(pi)*orientation(xbar), vq, angular_velocity(xbar)) 
end
settransform!(vis["robot"], Translation(position(xbar)) ∘ LinearMap(UnitQuaternion(orientation(xbar))))


In [103]:
linearization_angles = LinRange(0.0, 2*pi, 9)
quats_linearize = [[cos(α/2), 0, sin(α/2), 0] for α in linearization_angles[1:end-1]]
for quat in quats_linearize
    settransform!(vis[coordinate_frame_name], Translation(0, 0, 0.5) ∘ LinearMap(UnitQuaternion(quat)))
    sleep(0.5)
end

In [82]:
for i in 1:length(Xref)
    settransform!(vis[coordinate_frame_name], Translation(Xref[i][1:3]) ∘ LinearMap(UnitQuaternion(Xref[i][4:7])))
    sleep(0.01)
end

In [11]:
coordinate_frame_name = "solution_end_horizon_frame"

x_arrow_material = LineBasicMaterial(color=Colors.RGBA(0.55,0.1,0.1,1.0))
x_shaft = Cylinder(zero(Point{3, Float64}), Point(0.2, 0.0, 0.0), 0.01)
setobject!(vis[coordinate_frame_name][:xarrow][:shaft], x_shaft, x_arrow_material)
x_head = Cone(Point(0.2, 0.0, 0.0), Point(0.25, 0.0, 0.0), 0.025)
setobject!(vis[coordinate_frame_name][:xarrow][:head], x_head, x_arrow_material)

y_arrow_material = LineBasicMaterial(color=Colors.RGBA(0.1,0.55,0.1,1.0))
y_shaft = Cylinder(zero(Point{3, Float64}), Point(0.0, 0.2, 0.0), 0.01)
setobject!(vis[coordinate_frame_name][:yarrow][:shaft], y_shaft, y_arrow_material)
y_head = Cone(Point(0.0, 0.2, 0.0), Point(0.0, 0.25, 0.0), 0.025)
setobject!(vis[coordinate_frame_name][:yarrow][:head], y_head, y_arrow_material)

z_arrow_material = LineBasicMaterial(color=Colors.RGBA(0.1,0.1,0.55,1.0))
z_shaft = Cylinder(zero(Point{3, Float64}), Point(0.0, 0.0, 0.2), 0.01)
setobject!(vis[coordinate_frame_name][:zarrow][:shaft], z_shaft, z_arrow_material)
z_head = Cone(Point(0.0, 0.0, 0.2), Point(0.0, 0.0, 0.25), 0.025)
setobject!(vis[coordinate_frame_name][:zarrow][:head], z_head, z_arrow_material)

# delete!(vis[coordinate_frame_name])

MeshCat Visualizer with path /meshcat/solution_end_horizon_frame/zarrow/head at http://127.0.0.1:8702

In [87]:
include("../LibTinyMPC.jl")
x0 = X̃ref[:,1] + randn(nx)*0.0
Xhist = zeros(nx+1, NTOTAL)
Xhist[:,1] .= [x0[1:3]; rptoq(x0[4:6]); x0[7:12]]
Uhist = zeros(nu, NTOTAL-1)

for i = 1:NTOTAL-1
    # Update initial state
    solver.workspace.x[:,1] = [Xhist[:,i][1:3]; qtorp(Xhist[:,i][4:7]); Xhist[:,i][8:end]]

    # Update reference
    if i < NTOTAL - Nh
        solver.workspace.Xref = X̃ref[:,i:i+Nh-1]
    else
        solver.workspace.Xref = [X̃ref[:,i:end] repeat(X̃ref[:,end], 1, i+Nh-NTOTAL)]
    end

    # Solve
    solve_admm!(solver)
    Uhist[:,i] .= solver.workspace.u[:,1] + uhover

    # Simulate with control
    Xhist[:,i+1] = quad_dynamics_rk4(Xhist[:,i], Uhist[:,i], h)








    # Visualization stuff

    # Update quadrotor transform
    xbar = RBState(quad_model, Xhist[:,i])
    if quad_model.ned
        rq = position(xbar)
        vq = linear_velocity(xbar)
        rq = SA[rq[1],-rq[2],-rq[3]]
        vq = SA[vq[1],-vq[2],-vq[3]]
        xbar = RBState(rq, RotX(pi)*orientation(xbar), vq, angular_velocity(xbar)) 
    end
    settransform!(vis["robot"], Translation(position(xbar)) ∘ LinearMap(UnitQuaternion(orientation(xbar))))
    
    # Visualize solution
    delete!(vis["xLinearized"])
    Xsol = [solver.workspace.x[:,j] for j in 1:size(solver.workspace.x, 2)]
    x_linearized = [Point(x_[1], x_[2], x_[3]) for x_ in Xsol]
    setobject!(vis["xLinearized"], Object(PointCloud(x_linearized), 
    LineBasicMaterial(color=Colors.RGBA(1,0.6,0.0)), "Line"))

    # Visualize quaternion at end of solution via coordinate frame
    settransform!(vis[coordinate_frame_name], Translation(Xsol[end][1:3]) ∘ LinearMap(UnitQuaternion(rptoq(Xsol[end][4:6]))))

    # Visualize history
    xhist = [Xhist[:,j] for j in 1:i]
    delete!(vis["xHistline"])
    xHistline = [Point(x_[1], x_[2], x_[3]) for x_ in xhist]
    setobject!(vis["xHistline"], Object(PointCloud(xHistline), 
    LineBasicMaterial(color=Colors.RGBA(0.75,0.5,0.8)), "Line"))

    sleep(0.01)
end

In [145]:
for i in 1:size(Xhist, 2)
    settransform!(vis[coordinate_frame_name], Translation(Xhist[:,i][1:3]) ∘ LinearMap(UnitQuaternion(Xhist[:,i][4:7])))
    sleep(0.01)
end